# SBM assignment - Data Cleaning and Inspection

### Importing packages

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
import numpy as np

### Loading dataset

In [2]:
#Run the line below if you start with the full data set
#google_last_df = pd.read_csv("../Data/google_data_last.csv",encoding='cp1252')

#Run the line below if you start with the cleaned data set
cleaned_data = pd.read_csv("../cleaned.csv",encoding='cp1252')



In [3]:
#Run the line below if you start with the cleaned data set
data = cleaned_data.copy()

#Run the line below if you start with the full data set
data = google_last_df.copy()

In [10]:
data

,my_app_id,rating_app,nb_rating,num_downloads,price_gplay,has_ads,in_app,num_downloads_cat,more_from_developer,year_published,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,AOIC.AOIC_anb01,3.0,2.0,100.0,24.5 CZK,False,False,100 - 999,jp.aoi_lab.aoia_aoia_nixie,2012.0,...,0,0,0,0,0,0,0,0,0,0
1,ATIPIC.GUI,4.3,10.0,100.0,60 CZK,False,False,100 - 999,ATIPIC_EVAL.GUI,2015.0,...,0,0,0,0,0,0,0,0,0,0
2,Alekseyt.Lusher,4.3,8225.0,1000000.0,0,True,False,1000000 +,"com.Alekseyt.TronNet, com.Alekseyt.CircleMaze,...",2015.0,...,0,0,0,0,0,0,0,0,0,0
3,Alfasoft.Widgets.CroatiaFlagAnalogClock,5.0,1.0,10.0,30.99 CZK,False,False,0 - 99,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
4,Alfasoft.Widgets.FinlandFlagAnalogClock,4.0,1.0,10.0,30.99 CZK,False,False,0 - 99,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
5,Alfasoft.Widgets.ItalyFlagAnalogClock,4.7,6.0,100.0,30.99 CZK,False,False,100 - 999,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
6,Alfasoft.Widgets.MexicoFlagAnalogClock,4.8,10.0,100.0,30.99 CZK,False,False,100 - 999,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
7,Alfasoft.Widgets.PsychedelicAnalogClock,3.9,416.0,50000.0,0,False,False,10000 - 99999,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
8,Alfasoft.Widgets.RussiaFlagAnalogClock,5.0,3.0,50.0,32.39 CZK,False,False,0 - 99,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0
9,Alfasoft.Widgets.SouthKoreaFlagAnalogClock,5.0,1.0,10.0,32.39 CZK,False,False,0 - 99,"com.alfasoft.Widgets.TibetAnalogClock, com.alf...",2014.0,...,0,0,0,0,0,0,0,0,0,0


### Cleaning the data

In [11]:
#Filter variables

#run the line below if you start with the full dataset
#google_df = data[['my_app_id','rating_app','nb_rating', 'num_downloads', 'price_gplay', 'has_ads', 'in_app', 'categ_app', 'date_published', 'software_version', 'content_rating_app', 'more_from_developer']].copy()

#run the line below if you start with the cleaned dataset
google_df = data[['my_app_id','rating_app','nb_rating', 'num_downloads', 'price_gplay', 'has_ads', 'in_app', 'more_from_developer']].copy()


In [12]:
#looking at the NA's in rating and number of ratings
google_df[google_df.rating_app.isna()][['rating_app']].describe()
# Here we conclude that in most cases the missing values in rating_app & nb_rating are caused by no ratings being present so for RQ1 we can safely discard the missing values.
#Now we can look at the missing values in number of downloads.

#looking at number of downloads
google_df[google_df.num_downloads.isna()]
google_df[google_df.num_downloads.isna()][['nb_rating']].describe()
#We can see that a missing value in number of downloads often pairs with a missing value in rating, number of ratings and price. The datapoints with a missing number of downloads were either never downloaded, 
#or are a result of an error whilst gathering the data. We can never be sure about the first case, so we cannot convert them all to 0. Therefore we assume they are errors and delete these datapoints.

google_df = google_df.dropna(subset=['rating_app']) #comment voor goede dataset

In [17]:
#create year and month published
google_df['year_published'] = data.date_published.apply(lambda x: str(x)[-4:])
google_df['year_published'] = google_df['year_published'].replace({'span': 2012}, regex=True)
google_df['year_published'] = google_df['year_published'].astype(float)
google_df = google_df.drop('date_published', axis=1)
google_df['age'] = google_df.year_published.apply(lambda x: 2020 - x)

#content rating of app
google_df['content_rating_app'] = google_df.content_rating_app.apply(lambda x: str(x)[:7].replace(' ', '_'))
google_df.content_rating_app = google_df.content_rating_app.replace('nan', 'nan_content', regex = True)
OH_content = pd.get_dummies(google_df['content_rating_app'])
google_df = google_df.drop('content_rating_app', axis=1)
google_df = google_df.join(OH_content)
google_df = google_df.rename(columns = {'PEGI_7_':'PEGI_7'})

#number of new versions
google_df['version'] = data.software_version.apply(lambda x: str(x)[:1])
google_df = google_df.drop('software_version', axis=1)

#minimal app operating system
google_df['min_operating_system'] = data.operating_system.apply(lambda x: 'version_' + str(x)[:3])
google_df.min_operating_system = google_df.min_operating_system.replace('nan', 'nan_os', regex = True)
OH_OS = pd.get_dummies(google_df['min_operating_system'])
google_df = google_df.drop('min_operating_system', axis=1)
google_df = google_df.join(OH_OS)

AttributeError: 'DataFrame' object has no attribute 'date_published'

In [16]:




#number of apps per developer
google_df['nb_apps_developer'] = data.more_from_developer.apply(lambda x: 1 if type(x) == float else x.count(',') + 2)





#clean num_download
google_df.num_downloads = google_df.num_downloads.str.replace('+','')
google_df.num_downloads = google_df.num_downloads.str.replace('>','')
google_df.num_downloads = google_df.num_downloads.str.replace(' ','')
google_df.num_downloads = google_df.num_downloads.str.replace(',','')
google_df.num_downloads = google_df.num_downloads.astype('float')

#add a new column with the nr of downloads as a catagory 
category = pd.cut(google_df.num_downloads, bins=[0,99,999,9999,99999,999999, 99999999999], labels=['0 - 99','100 - 999','1000 - 9999','10000 - 99999', '100000 - 999999', '1000000 +'])
google_df.insert(8,'num_downloads_cat', category)

#Make new column with free vs paid boolean
zero_values = ['0', '0 USD', '0 SEK', '0 GBP', '0 MXN', '0 HKD', '0 KRW', '0 CZK', '0 CAD', '0 AED', '0 DKK', '0 IDR', '0 TWD', '0 INR', '0 CLP', '0 SAR', '0 ILS', '0 RUB', '0 AUD', '0 PLN','0 CHF', '0 MYR', '0 TRY', '0 BRL', '0 BGN', '0 UAH', '0 JPY', '0 EUR'] 
for value in zero_values:
    google_df['price_gplay'] = google_df['price_gplay'].replace([value],0)
google_df['free_app'] = google_df['price_gplay'] ==  0

'''
#Make a variable that has all possible catagories resulting from combinations of the booleans: free_app, has_ads and in_app
category = [     'free_noads_noinapp' if     google_df.loc[row].free_app and not google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'free_noads_inapp'   if     google_df.loc[row].free_app and not google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'free_ads_noinapp'   if     google_df.loc[row].free_app and     google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'free_ads_inapp'     if     google_df.loc[row].free_app and     google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'paid_noads_noinapp' if not google_df.loc[row].free_app and not google_df.loc[row].has_ads and not google_df.loc[row].in_app 
            else 'paid_noads_inapp'   if not google_df.loc[row].free_app and not google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'paid_ads_noinapp'   if not google_df.loc[row].free_app and     google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'paid_ads_inapp'     for row in google_df.index]
google_df['category'] = category
'''

#clean rating variable
google_df.nb_rating = google_df.nb_rating.str.replace(',','')
google_df.nb_rating = google_df.nb_rating.astype('float')

#create interaction variables
google_df['freeApp_x_hasAds'] = google_df['free_app'] & google_df['has_ads']
google_df['freeApp_x_inApp'] = google_df['free_app'] & google_df['in_app']
google_df['freeApp_x_hasAds_inApp'] = google_df['free_app'] & google_df['has_ads'] & google_df['in_app']

#turn catagorical variables into one-hot variables
OH_categ_app = pd.get_dummies(google_df['categ_app'])
google_df = google_df.drop('categ_app', axis=1)
google_df = google_df.join(OH_categ_app)

AttributeError: 'DataFrame' object has no attribute 'operating_system'

In [13]:
[value for value in google_df.columns]

['my_app_id',
 'rating_app',
 'nb_rating',
 'num_downloads',
 'price_gplay',
 'has_ads',
 'in_app',
 'num_downloads_cat',
 'more_from_developer',
 'year_published',
 'age',
 'PEGI_12',
 'PEGI_16',
 'PEGI_18',
 'PEGI_3',
 'PEGI_7',
 'Parenta',
 'Unrated',
 'nan_content',
 'nb_apps_developer',
 'version',
 'version_1.0',
 'version_1.1',
 'version_1.5',
 'version_1.6',
 'version_2.0',
 'version_2.1',
 'version_2.2',
 'version_2.3',
 'version_3.0',
 'version_3.1',
 'version_3.2',
 'version_4.0',
 'version_4.1',
 'version_4.2',
 'version_4.3',
 'version_4.4',
 'version_5.0',
 'version_5.1',
 'version_6.0',
 'version_7.0',
 'version_7.1',
 'version_8.0',
 'version_Var',
 'version_nan_os',
 'free_app',
 'freeApp_x_hasAds',
 'freeApp_x_inApp',
 'freeApp_x_hasAds_inApp',
 'Action',
 'Action & Adventure',
 'Adventure',
 'Arcade',
 'Art & Design',
 'Auto & Vehicles',
 'Beauty',
 'Board',
 'Books & Reference',
 'Brain Games',
 'Business',
 'Card',
 'Casino',
 'Casual',
 'Comics',
 'Communication',

# Wat er nog kan gebeuren aan features:
- Developer moet one-hot encoded maar 250.000 verschillende developers
- Age is nu aantal jaar, kan misschien aantal dagen worden
- Operating system pakt nu alleen eerte drie, dus bijv 4.0.3 word 4.0
- Operating system zou ook nog naar de maximale os kunnen kijken, dat doen we nu niet

### Save the file

In [14]:
google_df.to_csv('../Data/cleaned.csv',index=False)